In [1]:
import requests
import lxml 

from time import time
from bs4 import BeautifulSoup
from selectolax.parser import HTMLParser

from bs4 import BeautifulSoup, NavigableString, Tag

def html_to_text(html):
    "Creates a formatted text email message as a string from a rendered html template (page)"
    soup = BeautifulSoup(html, 'html.parser')
    # Ignore anything in head
    body, text = soup.body, []
    for element in body.descendants:
        # We use type and not isinstance since comments, cdata, etc are subclasses that we don't want
        if type(element) == NavigableString:
            parent_tags = (t for t in element.parents if type(t) == Tag)
            hidden = False
            for parent_tag in parent_tags:
                # Ignore any text inside a non-displayed tag
                # We also behave is if scripting is enabled (noscript is ignored)
                # The list of non-displayed tags and attributes from the W3C specs:
                if (parent_tag.name in ('area', 'base', 'basefont', 'datalist', 'head', 'link',
                                        'meta', 'noembed', 'noframes', 'param', 'rp', 'script',
                                        'source', 'style', 'template', 'track', 'title', 'noscript') or
                    parent_tag.has_attr('hidden') or
                    (parent_tag.name == 'input' and parent_tag.get('type') == 'hidden')):
                    hidden = True
                    break
            if hidden:
                continue

            # remove any multiple and leading/trailing whitespace
            string = ' '.join(element.string.split())
            if string:
                if element.parent.name == 'a':
                    a_tag = element.parent
                    # replace link text with the link
                    string = a_tag['href']
                    # concatenate with any non-empty immediately previous string
                    if (    type(a_tag.previous_sibling) == NavigableString and
                            a_tag.previous_sibling.string.strip() ):
                        text[-1] = text[-1] + ' ' + string
                        continue
                elif element.previous_sibling and element.previous_sibling.name == 'a':
                    text[-1] = text[-1] + ' ' + string
                    continue
                elif element.parent.name == 'p':
                    # Add extra paragraph formatting newline
                    string = '\n' + string
                text += [string]
    doc = '\n'.join(text)
    return doc


def get_text_bs(html):
    tree = BeautifulSoup(html, 'lxml')

    body = tree.body
    if body is None:
        return None

    for tag in body.select('script'):
        tag.decompose()
    for tag in body.select('style'):
        tag.decompose()

    text = body.get_text(separator='\n')
    return text


def get_text_selectolax(html):
    tree = HTMLParser(html)

    if tree.body is None:
        return None

    for tag in tree.css('script'):
        tag.decompose()
    for tag in tree.css('style'):
        tag.decompose()

    text = tree.body.text(separator='\n')
    return text

In [2]:
url = 'https://sambaiga.github.io/2017/05/03/hmm-intro.html'
html = requests.get(url).content

In [3]:
doc = get_text_bs(html)
doc

'\n\n\n\n\n\n\n\n\n  \n    \n  sambaiga\n\n  \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nMenu:\n\n\n\n\nAbout\n\n\nBlog\n\n\nProjects\n\n\nResources\n\n\nTalk\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nThe Basic of Hidden Markov Model\n\n\n\n\n\n          \n\n  May 3, 2017\n\n\n        \n\n\n·\n\n    \n\n    last updated on\n    \n\n  Oct 1, 2017\n\n\n\n    \n\n    \n\n\n\n\nThis post review basic of HMM and its implementation in Python.\n\n\n\n\n\n\n\n\n\nIntroduction\n\n\nHMM is a Markov model whose states are not directly observed instead each state is characterised by a probability distribution function modelling the observation corresponding to that state. HMM has been extensively used in temporal pattern recognition such as speech, handwriting, gesture recognition, robotics, biological sequences and recently in energy disaggregation. This tutorial will introduce the basic concept of HMM.\n\n\nThere are two variables in HMM: observed variables and hidden variables where 

    <h2 id="introduction">Introduction</h2>

<p>HMM is a Markov model whose states are not directly observed instead each state is characterised by a probability distribution function modelling the observation corresponding to that state. HMM has been extensively used in temporal pattern recognition such as speech, handwriting, gesture recognition, robotics, biological sequences and recently in energy disaggregation. This tutorial will introduce the basic concept of HMM.</p>

<p>There are two variables in HMM: observed variables and hidden variables where the sequences of hidden variables forms a Markov process as shown in figure below. In the context of NILM, the hidden variables are used to model states(ON,OFF, standby etc) of individual appliances and the observed variables are used to model the electric usage. HMMs has been widely used in most of the recently proposed NILM approach because it represents well the individual appliance internal states which are not directly observed in the targeted energy consumption.</p>
<figure>
  <img src="/assets/img/post/hmm.png" title="HMM graphical model" alt="">
  <figcaption>HMM graphical model
  </figcaption>
</figure>

<p>A typical HMM is characterised by the following:</p>

<ul>
  <li>The finite set of hidden states  <script type="math/tex">S</script> (e.g ON, stand-by, OFF, etc.) of an appliance,  <script type="math/tex">S = \{s_1, s_2....,s_N\}</script>.</li>
  <li>The finite set of  <script type="math/tex">M</script> observable symbol  <script type="math/tex">Y</script> per states (power consumption) observed in each state,  <script type="math/tex">Y = \{y_1, y_2....,y_M\}</script>. The observable symbol  <script type="math/tex">Y</script> can be discrete or a continuous set.</li>
  <li>The transition matrix   <script type="math/tex">\mathbf{A}=\{a_{ij},1\leq i,j \geq N\}</script> represents the probability of moving from state  <script type="math/tex">s_{t-1}=i</script> to  <script type="math/tex">s_t =j</script> such that:  <script type="math/tex">a_{ij} = P(s_{t} =j \mid s_{t-1}=i)</script>, with  <script type="math/tex">a_{ij} \leq 0</script> and where  <script type="math/tex">s_t</script> denotes the state occupied by the system at time  <script type="math/tex">t</script>. The matrix  <script type="math/tex">\mathbf{A}</script> is  <script type="math/tex">N x N</script>.</li>
  <li>The emission matrix  <script type="math/tex">\mathbf{B} =\{b_j(k)\}</script> representing the probability of emission of symbol  <script type="math/tex">k</script>  <script type="math/tex">\epsilon</script>  <script type="math/tex">Y</script> when system state is  <script type="math/tex">s_t=j</script> such that:  <script type="math/tex">b_j(k) = p(y_t = k  \mid  s_t=j)</script> The matrix  <script type="math/tex">\mathbf{B}</script> is an  <script type="math/tex">N x M</script>. The emission probability can be discrete or continous distribution. If the emission is descrete a multinomial distribution is used and multivariate Gaussian distribution is usually used for continous emission.</li>
  <li>And the initial state probability distribution  <script type="math/tex">\mathbf{\pi}  = \{\pi_i \}</script> indicating the probability of each state of the hidden variable  at  <script type="math/tex">t = 1</script> such that,  <script type="math/tex">\pi _i = P(q_1 = s_i), 1 \leq i \geq N</script>.</li>
</ul>

<p>The complete HMM specification requires;</p>

<ol>
  <li>Finite set of hidden states <script type="math/tex">N</script> and observation symbols <script type="math/tex">M</script>
</li>
  <li>Length of observation seqences <script type="math/tex">T</script> and</li>
  <li>Specification of three probability measures <script type="math/tex">\mathbf{A}, \mathbf{B}</script> and <script type="math/tex">\mathbf{\pi}</script>
</li>
</ol>

<p>The set of all HMM model parameters is represented by <script type="math/tex">\mathbf{\lambda} =(\pi, A, B)</script>.</p>

<p>Since <script type="math/tex">S</script> is not observed, the likelihood function <script type="math/tex">Y</script> is given by the joint distribution of <script type="math/tex">Y</script> and <script type="math/tex">S</script> over all possible state.</p>

<script type="math/tex; mode=display">P(Y \mid \lambda) = \sum P(Y, S \mid  \lambda)</script>

<p>where</p>

<script type="math/tex; mode=display">P(Y,S \mid \lambda) = P(Y \mid S,\lambda)P(S \mid \lambda)</script>

<p>Note that <script type="math/tex">y_t</script> is independent and identically distributed given state sequence <script type="math/tex">S = \{s_1, s_2....,s_N\}</script>. Also each state at time <script type="math/tex">t</script> depend on the state at its previous time <script type="math/tex">t-1</script>. Then</p>

<script type="math/tex; mode=display">P(Y \mid S, \lambda) = \prod_{t=1}^T P(y_t \mid s_t)</script>

<p>Similary</p>

<script type="math/tex; mode=display">P(S \mid \lambda) = \pi _{s_1} \prod _{t=2}^T a_{ij}</script>

<p>The joint probability is therefore:</p>

<script type="math/tex; mode=display">P(Y \mid \lambda) = \pi _{s_1}P(y_1 \mid s_1) \sum \prod_{t=2}^T a_{ij} P(y_t \mid s_t)</script>

<h2 id="three-main-problems-in-hmms">Three main problems in HMMs</h2>

<p>When applying HMM to a real world problem, three important problem must be solved.</p>

<ol>
  <li>Evaluation Problem: Given HMM parameters <script type="math/tex">\lambda</script> and the observation seqence <script type="math/tex">Y = \{Y_1, Y_2....,Y_M\}</script>, find <script type="math/tex">P(Y \mid \lambda)</script> the likelihood of the observation sequence <script type="math/tex">Y</script> given the model <script type="math/tex">\lambda</script>. This problem give a score on how well a given model matches a given observation and thus allows you to choose the model that best match the observation.</li>
  <li>Decoding Problem: Given HMM parameters <script type="math/tex">\lambda</script> and the observation seqence <script type="math/tex">Y = \{Y_1, Y_2....,Y_M\}</script>, find an optimal state sequense <script type="math/tex">S = \{S_1, S_2....,S_N\}</script> which best explain the observation.This problem attempt to cover the hidden part of the model.</li>
  <li>Learning Problem: Given the obseravtion seqence <script type="math/tex">Y = \{Y_1, Y_2....,Y_M\}</script>, find the model parameters <script type="math/tex">\lambda</script> that maximize <script type="math/tex">P(Y \mid \lambda)</script>.This problem attempt to optimize the model parameters so as to describe the model.</li>
</ol>

<p>The first and the second problem can be solved by the dynamic programming algorithms known as the Viterbi algorithm and the Forward-Backward algorithm, respectively. The last one can be solved by an iterative Expectation-Maximization (EM) algorithm, known as the Baum-Welch algorithm. We will discuss the first and the second problem in this post.</p>

<h2 id="solution-to-problem-1">Solution to Problem 1</h2>

<p>A straight forward way to solve this problem is to find <script type="math/tex">P(Y \mid S, \lambda)</script> for fixed state sequences <script type="math/tex">S = \{s_1,...s_T \}</script> and then sum up over all possible states. This is generally infeasible since it requires about <script type="math/tex">2TN^T</script> multiplications. However this problem can be efficiently solved by using the forward algorithm  as follows:</p>

<h3 id="the-forward-backward-algorithm">The forward-backward Algorithm</h3>

<p>Let us define the <strong>forward variable</strong></p>

<script type="math/tex; mode=display">\alpha _t(i)=P(y_1,\ldots y_t, s_t=i \mid \lambda)</script>

<p>the probability of the partial observation sequences <script type="math/tex">y_1 \ldots y_t</script>  up to time <script type="math/tex">t</script> and the state <script type="math/tex">s_t =i</script> at time <script type="math/tex">t</script> given the model <script type="math/tex">{\lambda}</script>. We also define an emission probability given HMM state <script type="math/tex">i</script> at time <script type="math/tex">t</script> as <script type="math/tex">b_i(y_t)</script>.</p>

<h4 id="forward-algorithm">Forward-Algorithm</h4>

<p><strong>Initilization</strong></p>

<p>Let</p>

<script type="math/tex; mode=display">% <![CDATA[
\begin{aligned}
\alpha _1(i)&=P(y_1, s_1=i \mid \lambda) \\
    & = P(y_1 \mid s_1=i,\lambda)P(s_1=i \mid \lambda)\\
    &= \pi _i b_i(y_1) \text{  for  } 1\leq i \geq N
\end{aligned} %]]></script>

<p><strong>Induction</strong></p>

<p>For <script type="math/tex">t=2,3...T</script> and <script type="math/tex">1\leq i \geq N</script>, compute:</p>

<script type="math/tex; mode=display">% <![CDATA[
\begin{aligned}
\alpha _{t}(i) & = P(y_1 \ldots y_t, s_t=i \mid \lambda)\\
 &= \displaystyle \sum_{j=1}^{N} P(y_1 \ldots y_{t}, s_{t-1}=j,s_t=i \mid \lambda) \\
 &= \displaystyle \sum_{j=1}^{N} P(y_t \mid s_t=i, y_1,\ldots y_{t-1}, s_{t-1}=j, \lambda) \\
   &  \times P(s_t=i \mid y_1 \ldots y_{t-1} \ldots , s_{t-1}=j, \lambda) \\
   & \times P(y_1 \ldots y_{t-1}, s_{t-1}=j,\lambda) \\
 & = P(y_t \mid s_t=i,\lambda)\displaystyle \sum_{j=1}^{N} P(s_t=i \mid s_{t-1}=j)\cdot P(y_1, \ldots y_{t-1}, s_{t-1}) \\
& = b_i(y_{t})\displaystyle \sum_{j=1}^{N} \alpha _{t-1}(i)a_{ij}  
\end{aligned} %]]></script>

<p><strong>Termination</strong></p>

<p>From <script type="math/tex">\alpha _t(i)=P(y_1,...y_t, s_t=i \mid \lambda)</script>, it cear that:</p>

<script type="math/tex; mode=display">% <![CDATA[
\begin{aligned} 
P(Y \mid \lambda) &= \displaystyle \sum_{i=1}^{N} P(y_1,\ldots y_T, s_T = i \mid \lambda) \\
&= \displaystyle \sum_{i=1}^{N}\alpha _T(i)  
\end{aligned} %]]></script>

<p>The forward algorithm only requires about <script type="math/tex">N^2T</script> multiplications and is it can be implemented in python as follows.</p>

<div class="language-python highlighter-rouge"><div class="highlight"><pre class="highlight"><code><span class="k">def</span> <span class="nf">forward</span><span class="p">(</span><span class="n">obs_seq</span><span class="p">):</span>
        <span class="n">T</span> <span class="o">=</span> <span class="nb">len</span><span class="p">(</span><span class="n">obs_seq</span><span class="p">)</span>
        <span class="n">N</span> <span class="o">=</span> <span class="n">A</span><span class="o">.</span><span class="n">shape</span><span class="p">[</span><span class="mi">0</span><span class="p">]</span>
        <span class="n">alpha</span> <span class="o">=</span> <span class="n">np</span><span class="o">.</span><span class="n">zeros</span><span class="p">((</span><span class="n">T</span><span class="p">,</span> <span class="n">N</span><span class="p">))</span>
        <span class="n">alpha</span><span class="p">[</span><span class="mi">0</span><span class="p">]</span> <span class="o">=</span> <span class="n">pi</span><span class="o">*</span><span class="n">B</span><span class="p">[:,</span><span class="n">obs_seq</span><span class="p">[</span><span class="mi">0</span><span class="p">]]</span>
        <span class="k">for</span> <span class="n">t</span> <span class="ow">in</span> <span class="nb">range</span><span class="p">(</span><span class="mi">1</span><span class="p">,</span> <span class="n">T</span><span class="p">):</span>
            <span class="n">alpha</span><span class="p">[</span><span class="n">t</span><span class="p">]</span> <span class="o">=</span> <span class="n">alpha</span><span class="p">[</span><span class="n">t</span><span class="o">-</span><span class="mi">1</span><span class="p">]</span><span class="o">.</span><span class="n">dot</span><span class="p">(</span><span class="n">A</span><span class="p">)</span> <span class="o">*</span> <span class="n">B</span><span class="p">[:,</span> <span class="n">obs_seq</span><span class="p">[</span><span class="n">t</span><span class="p">]]</span>
        <span class="k">return</span> <span class="n">alpha</span>

 <span class="k">def</span> <span class="nf">likelihood</span><span class="p">(</span><span class="n">obs_seq</span><span class="p">):</span>
        <span class="c1"># returns log P(Y  \mid  model)
</span>        <span class="c1"># using the forward part of the forward-backward algorithm
</span>        <span class="k">return</span>  <span class="n">forward</span><span class="p">(</span><span class="n">obs_seq</span><span class="p">)[</span><span class="o">-</span><span class="mi">1</span><span class="p">]</span><span class="o">.</span><span class="nb">sum</span><span class="p">()</span>      
</code></pre></div></div>

<h3 id="backward-algorithm">Backward Algorithm</h3>

<p>This is the same as the forward algorithm discussed in the previous sectionexcept that it start at the end and works backward toward the beginning. We first define the <strong>backward variable</strong> <script type="math/tex">\beta_t(i)=P(y_{t+1},y_{t+2} \ldots y_{T} \mid s_t=i, {\lambda})</script>: probability of the partial observed sequence from <script type="math/tex">t+1</script> to the end at <script type="math/tex">T</script> given state <script type="math/tex">i</script> at time <script type="math/tex">t</script> and the model <script type="math/tex">\lambda</script>.</p>

<p>Then <script type="math/tex">\beta_t(i)</script> can be computed recursively as follows.</p>

<p><strong>Initilization</strong></p>

<p>Let <script type="math/tex">\beta_{T}(i)= 1</script>, for <script type="math/tex">1 \leq i\geq N</script></p>

<p><strong>Induction</strong></p>

<p>For <script type="math/tex">t =T-1, T-2,\ldots1</script> for <script type="math/tex">1 \leq i\geq N</script> and by using the sum and product rules, we can rewrite <script type="math/tex">\beta_t(j)</script> as:</p>

<script type="math/tex; mode=display">% <![CDATA[
\begin{aligned}
\beta_t(i)&=P(y_{t+1},\ldots y_{T} \mid s_t=j, {\lambda}) \\
 &= \displaystyle \sum_{i=1}^{N} P(y_{t+1} \ldots y_T, s_{t+1}=i \mid s_t=j, \lambda) \\
 & = \displaystyle \sum_{i=1}^{N} P(y_{t+1} \ldots y_T, s_{t+1}=i, s_t=j, \lambda)\cdot P(s_{t+1}=i \mid s_t=j) \\
 &= \displaystyle \sum_{i=1}^{N} P(y_{t+2} \ldots y_T, s_{t+1}=i, \lambda)\cdot P(y_{t+1} \mid s_{t + 1}=i, \lambda)\cdot P(s_{t+1}=i \mid s_t=j) \\
 & = \displaystyle \sum_{i=1}^{N} a_{ij}b_i(y_{t+1})\beta _{t+1}(i)
\end{aligned} %]]></script>

<p><strong>Termination</strong></p>

<script type="math/tex; mode=display">% <![CDATA[
\begin{aligned}
\beta_{0} & = P(Y \mid \lambda) \\
& = \displaystyle \sum_{i=1}^{N} P(y_1,\ldots y_T, s_1=i) \\
&= \displaystyle \sum_{i=1}^{N} P(y_1,\ldots y_T \mid s_1=i)\cdot P(s_1=i) \\
& = \displaystyle \sum_{i=1}^{N} P(y_1 \mid s_1=i)\cdot P(y_2,\ldots y_T \mid s_1=i)\cdot P(s_1=i) \\
& = \displaystyle \sum_{i=1}^{N} \pi _i b_i(y_1)\beta _1(i)
\end{aligned} %]]></script>

<p>Python implementation of forward algorithm is as shown below;</p>

<div class="language-python highlighter-rouge"><div class="highlight"><pre class="highlight"><code><span class="k">def</span> <span class="nf">backward</span><span class="p">(</span><span class="n">obs_seq</span><span class="p">):</span>
        <span class="n">N</span> <span class="o">=</span> <span class="n">A</span><span class="o">.</span><span class="n">shape</span><span class="p">[</span><span class="mi">0</span><span class="p">]</span>
        <span class="n">T</span> <span class="o">=</span> <span class="nb">len</span><span class="p">(</span><span class="n">obs_seq</span><span class="p">)</span>

        <span class="n">beta</span> <span class="o">=</span> <span class="n">np</span><span class="o">.</span><span class="n">zeros</span><span class="p">((</span><span class="n">N</span><span class="p">,</span><span class="n">T</span><span class="p">))</span>
        <span class="n">beta</span><span class="p">[:,</span><span class="o">-</span><span class="mi">1</span><span class="p">:]</span> <span class="o">=</span> <span class="mi">1</span>

        <span class="k">for</span> <span class="n">t</span> <span class="ow">in</span> <span class="nb">reversed</span><span class="p">(</span><span class="nb">range</span><span class="p">(</span><span class="n">T</span><span class="o">-</span><span class="mi">1</span><span class="p">)):</span>
            <span class="k">for</span> <span class="n">n</span> <span class="ow">in</span> <span class="nb">range</span><span class="p">(</span><span class="n">N</span><span class="p">):</span>
                <span class="n">beta</span><span class="p">[</span><span class="n">n</span><span class="p">,</span><span class="n">t</span><span class="p">]</span> <span class="o">=</span> <span class="n">np</span><span class="o">.</span><span class="nb">sum</span><span class="p">(</span><span class="n">beta</span><span class="p">[:,</span><span class="n">t</span><span class="o">+</span><span class="mi">1</span><span class="p">]</span> <span class="o">*</span> <span class="n">A</span><span class="p">[</span><span class="n">n</span><span class="p">,:]</span> <span class="o">*</span> <span class="n">B</span><span class="p">[:,</span> <span class="n">obs_seq</span><span class="p">[</span><span class="n">t</span><span class="o">+</span><span class="mi">1</span><span class="p">]])</span>

        <span class="k">return</span> <span class="n">beta</span>
</code></pre></div></div>

<h4 id="posterior-probability">Posterior Probability</h4>
<p>The forward variable <script type="math/tex">\alpha _t(i)</script> and backward variable <script type="math/tex">\beta _t(i)</script> are used to calculate the posterior probability of a specific case. Now for <script type="math/tex">t=1...T</script> and <script type="math/tex">i=1..N</script>, let define posterior probability <script type="math/tex">\gamma_t(i)=P(s_t=i \mid Y, \lambda)</script> the probability of being in state <script type="math/tex">s_t = i</script> at time <script type="math/tex">t</script> given the observation <script type="math/tex">Y</script> and the model <script type="math/tex">\lambda</script>.</p>

<script type="math/tex; mode=display">% <![CDATA[
\begin{aligned}
\gamma_t(i) & = \frac{P(s_t=1, Y \mid \lambda)}{P(Y \mid \lambda)} \\
 &=\frac{P(y_1,\ldots y_t, s_t=1, \mid \lambda)}{P(Y \mid \lambda)}
\end{aligned} %]]></script>

<p>Consider:</p>

<script type="math/tex; mode=display">% <![CDATA[
\begin{aligned}
P(y_1,\ldots y_t, s_t=1, \mid \lambda) & = P(y_1,\ldots y_t \mid  s_t=1,\lambda)\cdot P(y_{t+1},\ldots y_T \mid  s_t=1,\lambda)\cdot P(s_t =i  \mid \lambda) \\
 & = \alpha _t(i) \cdot \beta _t(i)
\end{aligned} %]]></script>

<p>Thus</p>

<script type="math/tex; mode=display">\gamma_t(i) = \frac{\alpha _t(i) \cdot \beta _t(i)}{P(Y \mid \lambda)}</script>

<p>where</p>

<script type="math/tex; mode=display">P(Y \mid {\lambda}) =  \displaystyle \sum_{i=1}^{N}\alpha _T(i)</script>

<p>In python:</p>

<div class="language-python highlighter-rouge"><div class="highlight"><pre class="highlight"><code><span class="k">def</span> <span class="nf">gamma</span><span class="p">(</span><span class="n">obs_seq</span><span class="p">):</span>
    <span class="n">alpha</span> <span class="o">=</span> <span class="n">forward</span><span class="p">(</span><span class="n">obs_seq</span><span class="p">)</span>
    <span class="n">beta</span>  <span class="o">=</span> <span class="n">backward</span><span class="p">(</span><span class="n">obs_seq</span><span class="p">)</span>
    <span class="n">obs_prob</span> <span class="o">=</span> <span class="n">likelihood</span><span class="p">(</span><span class="n">obs_seq</span><span class="p">)</span>
    <span class="k">return</span> <span class="p">(</span><span class="n">np</span><span class="o">.</span><span class="n">multiply</span><span class="p">(</span><span class="n">alpha</span><span class="p">,</span><span class="n">beta</span><span class="o">.</span><span class="n">T</span><span class="p">)</span> <span class="o">/</span> <span class="n">obs_prob</span><span class="p">)</span>
</code></pre></div></div>

<p>We can use <script type="math/tex">\gamma_t(i)</script> to find the most likely state at time <script type="math/tex">t</script> which is the state <script type="math/tex">s_t=i</script> for which <script type="math/tex">\gamma_t(i)</script> is maximum. This algorithm <a href="http://www.shokhirev.com/nikolai/abc/alg/hmm/hmm.html">works fine in the case when HMM is ergodic</a> i.e. there is transition from any state to any other state. If applied to an HMM of another architecture, this approach could give a sequence that may not be a legitimate path because some transitions are not permitted. To avoid this problem <em>Viterbi algorithm</em> is the most common decoding algorithms used.</p>

<h3 id="viterbi-algorithm">Viterbi Algorithm</h3>

<p>Viterbi is a kind of dynamic programming algorithm that make uses of a dynamic programming trellis.</p>

<p>The virtebi algorithm offer an efficient way of finding  the single best state sequence.Let define the highest probability along a single path, at time <script type="math/tex">t</script>, which accounts for the first <script type="math/tex">t</script> observations and ends in state <script type="math/tex">j</script> using a new notation:</p>

<script type="math/tex; mode=display">% <![CDATA[
\begin{aligned}
\delta_t(i) & = \max_{s_1,\ldots s_{t-1}} P(s_1, \ldots s_t =1, y_1,\ldots y_t \mid \lambda)
\end{aligned} %]]></script>

<p>By induction, a recursive formula of <script type="math/tex">\delta_{t+1}(i)</script> from <script type="math/tex">\delta_t(i)</script>  is derived to calculate this probability as follows:</p>

<p>Consider the joint distribution appearing in <script type="math/tex">\delta_{t+1}(i)</script>, which can be rewritten when <script type="math/tex">s_{t+1}=i</script> and <script type="math/tex">s_t = j</script> as:</p>

<script type="math/tex; mode=display">% <![CDATA[
\begin{aligned}
P(s_1,\ldots, s_t=j,s_{t+1}=i, y_1,\ldots y_t, y_{t+1} \mid \lambda) & = P(s_1 \ldots s_t=j, y_1,\ldots y_t  \mid \lambda)\\& \times P(s_{t+1}=i,y_{t+1} \mid s_1, \ldots s_t, y_1, \ldots y_t, \lambda) \\
 & = P(s_1 \ldots s_t=j, y_1,\ldots y_t  \mid \lambda)\cdot P(s_{t+1} \mid s_t, \lambda)\\ & \times P(y_{t+1} \mid s_{t+1},\lambda) \\
  & = P(s_1 \ldots s_t=j, y_1,\ldots y_t  \mid \lambda)\cdot a_{ij}b_i(y_{t+1})
\end{aligned} %]]></script>

<p>Thus <script type="math/tex">\delta_{t+1}(i)</script>  is computed recursively from <script type="math/tex">\delta_{t+1}(j)</script> as:</p>

<script type="math/tex; mode=display">% <![CDATA[
\begin{aligned}
\delta_{t+1}(i) &= \max_{s_1,\ldots s_{t}=j} P(s_1 \ldots s_t=j, y_1,\ldots y_t  \mid \lambda)\cdot a_{ij}b_i(y_{t+1}) \\
 & = \max_{j}\Big[ \delta_t(j) a_{ij}\Big]\cdot b_i(y_{t+1})
\end{aligned} %]]></script>

<p>We therefore need to keep track the state that maximize the above equation so as to backtrack to the single best state sequence in the following Viterbi algorithm:</p>

<p><strong>Initilization</strong></p>

<p>For <script type="math/tex">1 \leq i \geq N</script>, let:</p>

<script type="math/tex; mode=display">% <![CDATA[
\begin{aligned}
\delta _1(i)&= \pi _{s_i}b_i(y_1)\\
\Theta _1(i)&=0
\end{aligned} %]]></script>

<p><strong>Recursion</strong></p>

<p>Calculate  the ML (maximum likelihood) state sequences and their probabilities. For <script type="math/tex">t=2,3,...T</script> and <script type="math/tex">1\leq i \geq N</script></p>

<script type="math/tex; mode=display">% <![CDATA[
\begin{aligned}\delta_t(i) & = \displaystyle \max_{j\epsilon{1,..N}} \Big[\delta_{t-1}(j)a_{ij}\Big]\cdot b_i(y_t) \\
\Theta_t(i) & = \arg\max_j \Big[\delta_{t-1}(j)a_{ij} \Big] 
\end{aligned} %]]></script>

<p><strong>Termination</strong>:</p>

<p>Retrieve the most likely final state</p>

<script type="math/tex; mode=display">% <![CDATA[
\begin{aligned} \hat{P} &= \displaystyle \max_{j\epsilon{1,..N}}[\delta_T(j)]  \\
\hat{S}_T & = \arg\max_j [\delta_T(j)]
\end{aligned} %]]></script>

<p><strong>State sequence backtracking</strong>:</p>

<p>Retrieve  the most likely state sequences (virtebi path)</p>

<script type="math/tex; mode=display">\hat{S}_t = \Theta_{t+1}(\hat{S}_{t+1}) \text{, where } t=T-1,T-2,\ldots1</script>

<p>Virtebi algorithm uses the same schema as the Forward algorithm except for two differences:</p>

<ol>
  <li>It uses maximization in place of summation at the recursion and termination steps.</li>
  <li>It keeps track of the arguments that maximize <script type="math/tex">\delta_t(i)</script> for each <script type="math/tex">t</script> and <script type="math/tex">i</script>, storing them in the N by T matrix <script type="math/tex">\Theta</script>. This matrix is used to retrieve the optimal state sequence at the backtracking step.</li>
</ol>

<p>Python implementation of virtebi algorithm</p>
<div class="language-python highlighter-rouge"><div class="highlight"><pre class="highlight"><code><span class="k">def</span> <span class="nf">viterbi</span><span class="p">(</span><span class="n">obs_seq</span><span class="p">):</span>
        <span class="c1"># returns the most likely state sequence given observed sequence x
</span>        <span class="c1"># using the Viterbi algorithm
</span>        <span class="n">T</span> <span class="o">=</span> <span class="nb">len</span><span class="p">(</span><span class="n">obs_seq</span><span class="p">)</span>
        <span class="n">N</span> <span class="o">=</span> <span class="n">A</span><span class="o">.</span><span class="n">shape</span><span class="p">[</span><span class="mi">0</span><span class="p">]</span>
        <span class="n">delta</span> <span class="o">=</span> <span class="n">np</span><span class="o">.</span><span class="n">zeros</span><span class="p">((</span><span class="n">T</span><span class="p">,</span> <span class="n">N</span><span class="p">))</span>
        <span class="n">psi</span> <span class="o">=</span> <span class="n">np</span><span class="o">.</span><span class="n">zeros</span><span class="p">((</span><span class="n">T</span><span class="p">,</span> <span class="n">N</span><span class="p">))</span>
        <span class="n">delta</span><span class="p">[</span><span class="mi">0</span><span class="p">]</span> <span class="o">=</span> <span class="n">pi</span><span class="o">*</span><span class="n">B</span><span class="p">[:,</span><span class="n">obs_seq</span><span class="p">[</span><span class="mi">0</span><span class="p">]]</span>
        <span class="k">for</span> <span class="n">t</span> <span class="ow">in</span> <span class="nb">range</span><span class="p">(</span><span class="mi">1</span><span class="p">,</span> <span class="n">T</span><span class="p">):</span>
            <span class="k">for</span> <span class="n">j</span> <span class="ow">in</span> <span class="nb">range</span><span class="p">(</span><span class="n">N</span><span class="p">):</span>
                <span class="n">delta</span><span class="p">[</span><span class="n">t</span><span class="p">,</span><span class="n">j</span><span class="p">]</span> <span class="o">=</span> <span class="n">np</span><span class="o">.</span><span class="nb">max</span><span class="p">(</span><span class="n">delta</span><span class="p">[</span><span class="n">t</span><span class="o">-</span><span class="mi">1</span><span class="p">]</span><span class="o">*</span><span class="n">A</span><span class="p">[:,</span><span class="n">j</span><span class="p">])</span> <span class="o">*</span> <span class="n">B</span><span class="p">[</span><span class="n">j</span><span class="p">,</span> <span class="n">obs_seq</span><span class="p">[</span><span class="n">t</span><span class="p">]]</span>
                <span class="n">psi</span><span class="p">[</span><span class="n">t</span><span class="p">,</span><span class="n">j</span><span class="p">]</span> <span class="o">=</span> <span class="n">np</span><span class="o">.</span><span class="n">argmax</span><span class="p">(</span><span class="n">delta</span><span class="p">[</span><span class="n">t</span><span class="o">-</span><span class="mi">1</span><span class="p">]</span><span class="o">*</span><span class="n">A</span><span class="p">[:,</span><span class="n">j</span><span class="p">])</span>

        <span class="c1"># backtrack
</span>        <span class="n">states</span> <span class="o">=</span> <span class="n">np</span><span class="o">.</span><span class="n">zeros</span><span class="p">(</span><span class="n">T</span><span class="p">,</span> <span class="n">dtype</span><span class="o">=</span><span class="n">np</span><span class="o">.</span><span class="n">int32</span><span class="p">)</span>
        <span class="n">states</span><span class="p">[</span><span class="n">T</span><span class="o">-</span><span class="mi">1</span><span class="p">]</span> <span class="o">=</span> <span class="n">np</span><span class="o">.</span><span class="n">argmax</span><span class="p">(</span><span class="n">delta</span><span class="p">[</span><span class="n">T</span><span class="o">-</span><span class="mi">1</span><span class="p">])</span>
        <span class="k">for</span> <span class="n">t</span> <span class="ow">in</span> <span class="nb">range</span><span class="p">(</span><span class="n">T</span><span class="o">-</span><span class="mi">2</span><span class="p">,</span> <span class="o">-</span><span class="mi">1</span><span class="p">,</span> <span class="o">-</span><span class="mi">1</span><span class="p">):</span>
            <span class="n">states</span><span class="p">[</span><span class="n">t</span><span class="p">]</span> <span class="o">=</span> <span class="n">psi</span><span class="p">[</span><span class="n">t</span><span class="o">+</span><span class="mi">1</span><span class="p">,</span> <span class="n">states</span><span class="p">[</span><span class="n">t</span><span class="o">+</span><span class="mi">1</span><span class="p">]]</span>
        <span class="k">return</span> <span class="n">states</span>
</code></pre></div></div>

<p>To summarize, we can compute the following from HMM:</p>

<ol>
  <li>The marginalized likelihood function <script type="math/tex">P(Y \mid \lambda)</script> from the forward or backward algorithm.</li>
  <li>The posterior probability <script type="math/tex">\gamma_t(i) = P(s_t=i  \mid Y, \lambda)</script> from the forward–backward algorithm.</li>
  <li>The optimal state sequence <script type="math/tex">\hat{S} = \max_{s} P(S \mid Y, \lambda) = \max_{s} P(S, Y \mid  \lambda)</script>from the Viterbi algorithm.</li>
  <li>The segmental joint likelihood function <script type="math/tex">P(\hat{S},Y \mid \lambda)</script> from the Viterbi algorithm.</li>
</ol>

<p>These values are used in the decoding step and the training step of estimating model parameters <script type="math/tex">\lambda</script>.</p>

<p><strong>Example 1</strong></p>

<p>Conside the Bob-Alice example as described <a href="https://en.wikipedia.org/wiki/Hidden_Markov_model#A_concrete_example">here</a>. Two friends, Alice and Bob, who live far apart from each other and who talk together daily over the telephone about what they did that day. Bob is only interested in three activities: walking in the park, shopping, and cleaning his apartment. The choice of what to do is determined exclusively by the weather on a given day. Alice has no definite information about the weather where Bob lives, but she knows general trends. Based on what Bob tells her he did each day, Alice tries to guess what the weather must have been like.</p>

<p>Alice believes that the weather operates as a discrete Markov chain. There are two states, “Rainy” and “Sunny”, but she cannot observe them directly, that is, they are hidden from her. On each day, there is a certain chance that Bob will perform one of the following activities, depending on the weather: “walk”, “shop”, or “clean”. Since Bob tells Alice about his activities, those are the observations.</p>

<div class="language-python highlighter-rouge"><div class="highlight"><pre class="highlight"><code><span class="n">states</span> <span class="o">=</span> <span class="p">(</span><span class="s">'Rainy'</span><span class="p">,</span> <span class="s">'Sunny'</span><span class="p">)</span>
<span class="n">observations</span> <span class="o">=</span> <span class="p">(</span><span class="s">'walk'</span><span class="p">,</span> <span class="s">'shop'</span><span class="p">,</span> <span class="s">'clean'</span><span class="p">)</span>
<span class="n">pi</span> <span class="o">=</span> <span class="n">np</span><span class="o">.</span><span class="n">array</span><span class="p">([</span><span class="mf">0.6</span><span class="p">,</span> <span class="mf">0.4</span><span class="p">])</span>  <span class="c1">#initial probability 
</span><span class="n">A</span> <span class="o">=</span> <span class="n">np</span><span class="o">.</span><span class="n">array</span><span class="p">([[</span><span class="mf">0.7</span><span class="p">,</span> <span class="mf">0.3</span><span class="p">],[</span><span class="mf">0.4</span><span class="p">,</span> <span class="mf">0.6</span><span class="p">]])</span> <span class="c1">#Transmission probability 
</span><span class="n">B</span> <span class="o">=</span> <span class="n">np</span><span class="o">.</span><span class="n">array</span><span class="p">([[</span><span class="mf">0.1</span><span class="p">,</span> <span class="mf">0.4</span><span class="p">,</span> <span class="mf">0.5</span><span class="p">],[</span><span class="mf">0.6</span><span class="p">,</span> <span class="mf">0.3</span><span class="p">,</span> <span class="mf">0.1</span><span class="p">]])</span> <span class="c1">#Emission probability
</span></code></pre></div></div>

<p>Suppose Bob says walk, clean, shop, shop, clean, walk. What will Alice hears.</p>

<div class="language-python highlighter-rouge"><div class="highlight"><pre class="highlight"><code><span class="n">bob_says</span> <span class="o">=</span> <span class="n">np</span><span class="o">.</span><span class="n">array</span><span class="p">([</span><span class="mi">0</span><span class="p">,</span> <span class="mi">2</span><span class="p">,</span> <span class="mi">1</span><span class="p">,</span> <span class="mi">1</span><span class="p">,</span> <span class="mi">2</span><span class="p">,</span> <span class="mi">0</span><span class="p">])</span>
<span class="n">alice_hears</span><span class="o">=</span><span class="n">viterbi</span><span class="p">(</span><span class="n">bob_says</span><span class="p">)</span>
<span class="k">print</span><span class="p">(</span><span class="s">"Bob says:"</span><span class="p">,</span> <span class="s">", "</span><span class="p">,</span><span class="nb">list</span><span class="p">(</span><span class="nb">map</span><span class="p">(</span><span class="k">lambda</span> <span class="n">y</span><span class="p">:</span> <span class="n">observ_bob</span><span class="p">[</span><span class="n">y</span><span class="p">],</span> <span class="n">bob_says</span><span class="p">)))</span>
<span class="k">print</span><span class="p">(</span><span class="s">"Alice hears:"</span><span class="p">,</span> <span class="s">", "</span><span class="p">,</span> <span class="nb">list</span><span class="p">(</span><span class="nb">map</span><span class="p">(</span><span class="k">lambda</span> <span class="n">s</span><span class="p">:</span> <span class="n">states_bob</span><span class="p">[</span><span class="n">s</span><span class="p">],</span> <span class="n">alice_hears</span><span class="p">)))</span>
</code></pre></div></div>

<blockquote>
    ('Bob says:', 'walk, clean, shop, shop, clean, walk')
    ('Alice hears:', 'Sunny, Rainy, Rainy, Rainy, Rainy, Sunny')
</blockquote>

<p>The notebook with codes for the above example can be found in <a href="https://github.com/sambaiga/HMM/blob/master/HMM%20Basics.ipynb">here</a></p>
